### 데이터 분석
    - 서울시 공원
        - 자치구별 공원면적
        - 단계 구분도
        - 자치구별 인당 공원면적
        - 단계구분도

- 경기도
    - 시군구별 공원 면적
    - 단계구분도
    - 시군구별 인당 공원 면적
    - 단계구분도

In [1]:
import numpy as np
import json,folium
import pandas as pd
import folium

In [2]:
df = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 non-null    

In [3]:
df = df[['공원명','면적','공원주소','지역']]
df.columns =['공원명','면적','주소','지역']
df.head(10)

,공원명,면적,주소,지역
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),중구
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),강동구
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,과천시
3,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가),성동구
4,월드컵공원,2284085㎡,서울특별시 마포구 하늘공원로 84(월드컵공원),마포구
5,광화문시민열린마당,8948.1㎡,서울특별시 종로구 세종로 76-2,종로구
6,진관근린공원(구파발폭포),휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 1...,서울특별시 은평구 진관동 393,은평구
7,중마루근린공원,6456㎥,서울특별시 영등포구 영등포동2가 222,영등포구
8,응봉공원,80309㎡ 배수지 시설용량 : 201.000㎥ 송수 펌프용량 : 직송급수계통 30...,서울특별시 성동구 금호동1가 산1-72,성동구
9,매화근린공원,1980.4㎡,서울특별시 강서구 등촌동 630-14,강서구


In [4]:
df.dropna(subset=['면적'],inplace=True)
df.isna().sum().sum()

1

In [5]:
df.면적[6]
df.drop([6],inplace=True)

In [6]:
import re
re.split('[,.]','a,b,c')

['a', 'b', 'c']

In [7]:
df.면적 =df.면적.str.replace('총','')
df.면적 = df.면적.apply(lambda x : re.split('[㎡m㎥]',x)[0])
df.면적 =df.면적.astype(float).astype(int)
df.면적.head()

0    2896887
1      80683
2    9132690
3     480994
4    2284085
Name: 면적, dtype: int32

In [8]:
df= df.groupby('지역')['면적'].sum().reset_index()
df.head()

,지역,면적
0,강남구,3626398
1,강동구,1526387
2,강북구,3714902
3,강서구,2194152
4,과천시,9132690


In [9]:
df = df.set_index('지역')
df     

,면적
지역,
강남구,3626398
강동구,1526387
강북구,3714902
강서구,2194152
과천시,9132690
관악구,10431453
광진구,584002
구로구,1617657
금천구,57568


In [10]:
df.drop(['과천시'],inplace=True)

In [11]:
file='data/seoul_geo_simple.json'
geo_data=json.load(open(file,encoding='utf-8'))

In [12]:
import numpy as np
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [13]:
gu_dict = get_text_location(geo_data)
gu_dict

{'강동구': [37.546532751824586, 127.14078218193268],
 '송파구': [37.50620055459428, 127.11197246712628],
 '강남구': [37.49679934694853, 127.0673229896189],
 '서초구': [37.47066332386016, 127.0390912771612],
 '관악구': [37.46393155080159, 126.94996289929526],
 '동작구': [37.495988209152785, 126.9530748356924],
 '영등포구': [37.51883998627326, 126.91028348590825],
 '금천구': [37.46241523438773, 126.9050257773362],
 '구로구': [37.49166908254099, 126.85870943441728],
 '강서구': [37.55703289257873, 126.83102419541086],
 '양천구': [37.529418412364095, 126.85252712500812],
 '마포구': [37.5599165285085, 126.9176866353821],
 '서대문구': [37.57597915089086, 126.94816079153975],
 '은평구': [37.61445865754635, 126.93220977883982],
 '노원구': [37.651705676071316, 127.07821750766884],
 '도봉구': [37.66711228271955, 127.03387331602003],
 '강북구': [37.646476408369075, 127.01356807286824],
 '성북구': [37.61007782092726, 127.02020916322154],
 '중랑구': [37.59847817045754, 127.09036092595524],
 '동대문구': [37.57621738848033, 127.04968775012617],
 '광진구': [37.548640

In [14]:
map = folium.Map([37.55,126.98],zoom_start=11,tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.면적,                 #단계구분도로 보여줄 데이터
    columns=[df.index,df.면적],   #데이터프레임에서 추출할 항목
    fill_color='YlOrRd',                #Colormap
    key_on='feature.id'               #지도에서 조인할 항목
).add_to(map)
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 공원 면적</h3>' # 지도 제목달기
map.get_root().html.add_child(folium.Element(title_html))
map

  - 자치구별 인당 공원면적
   - 단계구분도

In [15]:
import numpy as np
import json,folium
import pandas as pd
import folium

In [16]:
pop = pd.read_csv('../02.Pandas/과제/pop.csv')
pop.head(3)

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,153789,144683,9106,27818,5.92,18.09
1,중구,131787,122499,9288,24392,7.05,18.51
2,용산구,237285,222953,14332,39070,6.04,16.47


In [17]:
pop=pop[['구별','인구수']]
pop.columns=['지역','총인구']
pop.set_index('지역',inplace=True)
pop

,총인구
지역,
종로구,153789
중구,131787
용산구,237285
성동구,292672
광진구,352627
동대문구,352006
중랑구,391885
성북구,440142
강북구,302563


In [18]:
park = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
park = park[['면적','지역']]
park.head(10)

,면적,지역
0,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,중구
1,80683㎡,강동구
2,9132690m²,과천시
3,480994㎡,성동구
4,2284085㎡,마포구
5,8948.1㎡,종로구
6,휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 1...,은평구
7,6456㎥,영등포구
8,80309㎡ 배수지 시설용량 : 201.000㎥ 송수 펌프용량 : 직송급수계통 30...,성동구
9,1980.4㎡,강서구


In [19]:
park.dropna(subset=['면적'],inplace=True)
park.drop([6],inplace=True)

In [20]:
import re
re.split('[,.]','a,b,c')

['a', 'b', 'c']

In [21]:
park.면적 =park.면적.str.replace('총','')
park.면적 = park.면적.apply(lambda x : re.split('[㎡m㎥]',x)[0])
park.면적 =park.면적.astype(float).astype(int)
park.groupby('지역')['면적'].sum().reset_index()


,지역,면적
0,강남구,3626398
1,강동구,1526387
2,강북구,3714902
3,강서구,2194152
4,과천시,9132690
5,관악구,10431453
6,광진구,584002
7,구로구,1617657
8,금천구,57568
9,노원구,12024307


In [22]:
park=park.set_index('지역')
park.head()

,면적
지역,
중구,2896887
강동구,80683
과천시,9132690
성동구,480994
마포구,2284085


In [23]:
park.drop(['과천시'],inplace=True)
park.head()

,면적
지역,
중구,2896887
강동구,80683
성동구,480994
마포구,2284085
종로구,8948


In [34]:
park=park.groupby('지역')[['면적']].sum()
park

,면적
지역,
강남구,3626398
강동구,1526387
강북구,3714902
강서구,2194152
관악구,10431453
광진구,584002
구로구,1617657
금천구,57568
노원구,12024307


In [39]:
df = park.merge(pop,left_index=True,right_index=True)
df.head()

,면적,총인구
지역,,
강남구,3626398,537800
강동구,1526387,466472
강북구,3714902,302563
강서구,2194152,579768
관악구,10431453,499449


In [40]:
df['인당공원']=(df.총인구 / df.면적 *100 ).round(2)
df.head()

,면적,총인구,인당공원
지역,,,
강남구,3626398,537800,14.83
강동구,1526387,466472,30.56
강북구,3714902,302563,8.14
강서구,2194152,579768,26.42
관악구,10431453,499449,4.79


In [41]:
import numpy as np
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [42]:
gu_dict = get_text_location(geo_data)
gu_dict

{'강동구': [37.546532751824586, 127.14078218193268],
 '송파구': [37.50620055459428, 127.11197246712628],
 '강남구': [37.49679934694853, 127.0673229896189],
 '서초구': [37.47066332386016, 127.0390912771612],
 '관악구': [37.46393155080159, 126.94996289929526],
 '동작구': [37.495988209152785, 126.9530748356924],
 '영등포구': [37.51883998627326, 126.91028348590825],
 '금천구': [37.46241523438773, 126.9050257773362],
 '구로구': [37.49166908254099, 126.85870943441728],
 '강서구': [37.55703289257873, 126.83102419541086],
 '양천구': [37.529418412364095, 126.85252712500812],
 '마포구': [37.5599165285085, 126.9176866353821],
 '서대문구': [37.57597915089086, 126.94816079153975],
 '은평구': [37.61445865754635, 126.93220977883982],
 '노원구': [37.651705676071316, 127.07821750766884],
 '도봉구': [37.66711228271955, 127.03387331602003],
 '강북구': [37.646476408369075, 127.01356807286824],
 '성북구': [37.61007782092726, 127.02020916322154],
 '중랑구': [37.59847817045754, 127.09036092595524],
 '동대문구': [37.57621738848033, 127.04968775012617],
 '광진구': [37.548640

In [43]:
map = folium.Map([37.55,126.98],zoom_start=11,tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.인당공원,                
    columns=[df.index,df.인당공원],  
    fill_color='YlOrRd',                
    key_on='feature.id'               
).add_to(map)
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 공원 면적</h3>' # 지도 제목달기
map.get_root().html.add_child(folium.Element(title_html))
map